# Recipes Analysis

This solves the functional problem of maximizing efficiency of systems, by deciding,
the output rate of each recipe, after deciding that the preceding ingredients can be provided
at exactly a certain rate.

Post a recent update, the ingredients changed to having a multi-dimensional relation to a linear one, reportedly.

The aim is use some basic 'facts' in order to derive the rest.

In [1]:
# Facts

# Assume all rates are in items/min

conveyor_speeds = [60, 120, 270, 480, 780] # items/min
# belts and lifts have the same semantics

# All resources have a cap for miner mk3 at 250%
# extraction rate from the miner is capped by conveyor belts
# The minables.csv lists the cap
# The manufacturables.csv lists the recipes


In [2]:
import os
import math
import json
import pandas
manufacturables = pandas.read_csv('res/data/personal_collection/resources/manufacturables.csv')
buildings = pandas.read_csv('res/data/personal_collection/buildings/production/production_buildings.csv')
minables = pandas.read_csv('res/data/personal_collection/resources/minables.csv')

In [3]:
def list_all_materials():
    global manufacturables
    df = manufacturables
    arr = []
    arr += df['prod_1'].dropna().tolist()
    arr += df['prod_2'].dropna().tolist()
    arr += df['ing_1'].dropna().tolist()
    arr += df['ing_2'].dropna().tolist()
    arr += df['ing_3'].dropna().tolist()
    arr += df['ing_4'].dropna().tolist()
    arr = set(arr)
    return arr
# list_all_materials()

In [4]:
def material_exists(material='adaptive-control-unit'):
    if isinstance(file_or_arr, str):
        materials_list = list_all_materials()
    if material in materials_list:
        return True
    else:
        res = []
        for matter in materials_list:
            if material in matter:
                res.append(matter)
        return res
# material_exists('oil')

In [5]:
def query_ingredients(ingredient='ai-limiter'):
    global manufacturables
    df = manufacturables
    arr = []
    arr.append(df[df['ing_1'].str.contains(ingredient, na=False)])
    arr.append(df[df['ing_2'].str.contains(ingredient, na=False)])
    arr.append(df[df['ing_3'].str.contains(ingredient, na=False)])
    arr.append(df[df['ing_4'].str.contains(ingredient, na=False)])
    return pandas.concat(arr)
    
def query_products(df, product='ai-limiter'):
    arr = []
    arr.append(df[df['prod_1'].str.contains(product, na=False)])
    arr.append(df[df['prod_2'].str.contains(product, na=False)])
    return pandas.concat(arr)
query_products(manufacturables, product='ai-limiter')

,name,building,prod_1,prod_rate_1,prod_unit_1,prod_2,prod_rate_2,prod_unit_2,ing_1,rate_1,unit_1,ing_2,rate_2,unit_2,ing_3,rate_3,unit_3,ing_4,rate_4,unit_4
41,ai limiter,assembler,ai-limiter,5.0,items/min,NaN,NaN,NaN,copper-sheet,25.0,items/min,quickwire,100.0,items/min,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_mining_stat(item_name, rate):
    global minables, buildings
    resp = {
        'item' : item_name,
        'least_energy' : 1,
        'rate' : rate,
        'building' : 0,
        'machine_count' : 0
    }
    transforms = {
        'miner' : 'miner-mk3',
        'resource-well' : 'resource-well-pressurizer'
    }
    # print(minables[minables['name'] == item_name].iloc[0][1])
    minable = minables[(minables['name'] == item_name)]
    # print(minable)
    building = minable.iloc[0][1]
    building = building if building not in transforms.keys() else transforms[building]
    resp['building'] = building
    single_node_rate = minable.iloc[0][3]
    try:
        test = minable.iloc[-1][5]
        if not pandas.isna(test):
            single_node_rate = test
        else:
            test = minable.iloc[-1][3]
            if not pandas.isna(test):
                single_node_rate = test
    except:
        pass
    builds = math.ceil(rate / single_node_rate)
    resp['machine_count'] = builds
    # print(buildings[buildings['name'] == building])
    building_df = buildings[buildings['name'] == building]
    resp['least_energy'] = int(building_df.iloc[0][4]) * builds
    return resp

def get_recipe_stats(product, rate, max_ingredients = 4, max_products = 2, visited = [], depth = 5, best_only = True):
    global buildings, minables, manufacturables
    df = manufacturables
    least_energy, best_recipe = 99999, None
    recipe_arr = []
    resp = {
        'item' : product,
        'least_energy' : least_energy,
        'recipes' : recipe_arr
    }
    if depth < 0 :
        return resp
    recipes = query_products(df, product)
    for _, row in recipes.iterrows():
        if row['name'] in visited:
            continue
        visited_new = visited + [row['name']]
        # print(visited_new)
        res = {}
        res['name'] = row['name']
        res['building'] = row['building']
        res['rate'] = float(rate)
        prod_ctr = 1
        for i in range(1, max_products + 1):
            if row['prod_'+str(i)] == product:
                prod_ctr = i
        res['machine_count'] = math.ceil(rate / row['prod_rate_'+str(prod_ctr)])
        building_energy = math.ceil(buildings[buildings['name'] == res['building']].iloc[0][4] * res['machine_count'])
        ingredient_energy = 0
        res['ingredients'] = []
        for i in range(1, max_ingredients + 1):
            ingredient = row['ing_'+str(i)]
            if ingredient is None or pandas.isna(ingredient):
                break
            if ingredient in minables['name'].unique():
                ingredient = get_mining_stat(ingredient, float(row['rate_'+str(i)]) * res['machine_count'])
            else:
                ingredient = get_recipe_stats(ingredient, float(row['rate_'+str(i)]) * res['machine_count'], max_ingredients = max_ingredients, max_products = max_products, visited=visited_new, depth=depth - 1, best_only=best_only)
            ingredient_energy += ingredient['least_energy']
            res['ingredients'].append(ingredient)
        energy_cost = ingredient_energy + building_energy
        if energy_cost < least_energy:
            least_energy = energy_cost
            if best_only:
                best_recipe = res
        if not best_only:
            recipe_arr.append(res)
    if best_only:
        recipe_arr.append(best_recipe)
    
    if len(resp['recipes']) > 0 :
        resp['least_energy'] = least_energy
    else:
        resp['least_energy'] = 0
    return resp

def generate_recipe_json(ingredient, rate=60, depth = 5, indent=None, best_only=True, output_dir = os.path.join('output', 'recipes')):
    recipe = get_recipe_stats(ingredient, rate, depth=depth, best_only=best_only)
    try:
        os.makedirs(output_dir)
    except:
        pass
    filepath = os.path.join(output_dir, ingredient+'.json')
    fp = open(filepath, 'w')
    json.dump(recipe, fp, indent=indent)
    fp.close()

In [7]:
# generate_recipe_json('pressure-conversion-cube', depth = 7, indent = 2)

In [8]:
def get_ingredient_buildings(ing_dict):
    buildings = []
    if 'recipes' in ing_dict.keys():
        recipe = ing_dict['recipes'][0] # use only the first
        current_building = [recipe['building'], recipe['machine_count'], recipe['name'], recipe['rate'] / recipe['machine_count']]
        buildings.append(current_building)
        if 'ingredients' in recipe.keys():
            for ingredient in recipe['ingredients']:
                buildings += get_ingredient_buildings(ingredient)
    else:
        current_building = [ing_dict['building'], ing_dict['machine_count'], ing_dict['item'], ing_dict['rate'] / ing_dict['machine_count']]
        buildings.append(current_building)
    return buildings
        
def generate_recipe_buildings_list(ingredient, recipes_loc = os.path.join('output', 'recipes'), output_path = os.path.join('output', 'recipes')):
    filepath = os.path.join(recipes_loc, ingredient+'.json')
    fp = open(filepath, 'r')
    ing_details = json.load(fp)
    fp.close()
    summary = {'item' : ing_details['item'], 'energy' : ing_details['least_energy'], 'buildings' : get_ingredient_buildings(ing_details)}
    filepath = os.path.join(output_path, ingredient+'_buildings.json')
    fp = open(filepath, 'w')
    json.dump(summary, fp, indent = 2)
    fp.close()
    return summary

In [9]:
summary = generate_recipe_buildings_list('pressure-conversion-cube')

In [32]:
df = pandas.DataFrame(summary['buildings'], columns=['building', 'count', 'recipe', 'rate'])
# df = df[df['building'].isin(['miner-mk3', 'oil-extractor', 'water-extractor', 'resource-well-pressurizer'])]
df = df.groupby(['building','recipe'], as_index=False)['rate'].sum()
for idx, row in df.iterrows():
    bname = row['building']
    details = buildings[buildings['name'] == bname].iloc[0]
    brate = details.iloc[3]
    benergy = details.iloc[4]
    if 'resource-well' in bname:
        brate = buildings[buildings['name'] == 'resource-well-extractor'].iloc[0][3]
        benergy = buildings[buildings['name'] == 'resource-well-pressurizer'].iloc[0][4]
    
    count = math.ceil(row['rate'] / brate)
    df.loc[idx, 'building_count'] = count
    df.loc[idx, 'energy'] = count * benergy
df

,building,recipe,rate,building_count,energy
0,assembler,ai limiter,4.583333,-4.0,-60.0
1,assembler,alt alternate: alclad casing,110.454545,-110.0,-1650.0
2,assembler,alt encased industrial pipe,3.966346,-3.0,-45.0
3,assembler,alt silicon circuit board,12.272727,-12.0,-180.0
4,assembler,alt steeled frame,3.000000,-3.0,-45.0
5,assembler,alt stitched iron plate,5.500000,-5.0,-75.0
6,assembler,pressure conversion cube,1.000000,-1.0,-15.0
7,blender,alt heat-fused frame,3.000000,-3.0,-225.0
8,blender,nitric acid,30.000000,-30.0,-2250.0
9,constructor,alt caterium wire,107.142857,-107.0,-428.0


## Plant Building

The entire automated plant needs to accomodate the following : 

1. Energy for producing the energy itself
2. By-Products of the recipe, and how to handle them

This requires extremely expensive algorithms that works with calculating alternatives.
Especially for handling the by-products.

By-Product handling will require the system to understand what a good base material
that can be derived from the by product would be like.

It's going to be an exploratory work, which will again require the system to look for recipes for it to build.

Only after that completes, can the plant be genuinely considered to be self sustaining.

In [ ]:
def build_plant_schematics(ingredient, buildings_dir=os.path.join('output', 'recipes'), output_path = os.path.join('output', 'plants')):
    filepath = os.path.join(output_path, ingredient+'_buildings.json')
    fp = open(filepath, 'r')
    ing_details = json.load(fp)
    buildings = []
    buildings += get_energy_buildings(ing_details['energy'])
    